In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import scipy.io
import itertools as it
import scipy.special as psi
plt.style.use('classic')
import seaborn as sns
import pandas as pd
import math as mt
import time
import sys

sys.path.insert(1, '/rds/general/user/lr4617/home/4th_Year_Project/Final_Year_Project/')

from scipy.io import loadmat
from scipy import stats
from numpy.random import seed
from numpy.random import rand
from scipy.integrate import quad
from scipy.io import savemat
from tempfile import TemporaryFile
from scipy.io import loadmat
from sklearn.decomposition import PCA
from sklearn.manifold import MDS
from sklearn.decomposition import KernelPCA
from mpl_toolkits import mplot3d
from mPE_fn import mPE_
from scipy.spatial import distance
from scipy.stats import entropy
from mPE_ultis import integrand, ubble, array_list, permutation

In [14]:
##################################### CHANGE TRAJECTORY NUMBER HERE #####################################
traj_n = 8
#########################################################################################################

##################################### CHANGE TRAJECTORY MODE HERE #######################################
mode = 'drug'
#########################################################################################################

In [15]:
# load entire high-dimensional trajectory
path = '/rds/general/user/lr4617/home/4th_Year_Project/CAPTURE_rat_multidimensional/raw_data/' + str(mode) + '/traj_' + str(traj_n) + '/trajectories_na/'
trajectories = os.listdir(path)
f_s = 300
# removing NaN columns
nan_cols = []
for i, time_bin in enumerate(trajectories):
    if time_bin != 'behavs':
        trajectory = loadmat(path + time_bin)
        trajectory = trajectory['trajectory'] 
        for i in range(trajectory.shape[1]):
            if np.isnan(trajectory[:, i]).all():
                nan_cols.append(i)

In [16]:
def rolling_mean(x, window, overlapping=True):
    '''
    input:
        x      - input sequence
        window - rolling window
        
    returns:
        y      - moving averaged sequence
    '''
        
    ## assuming that there are more observations than variables:
    if np.max(x.shape)> x.shape[0]:
        x=x.transpose()
        
    y = np.zeros((x.shape))
    for i in range(0, x.shape[0] - window):
        y[i, :] = np.mean(x[i:i+window, :], axis=0)
        
    y[x.shape[0]-window:x.shape[0]-1, :] = x[x.shape[0]-window:x.shape[0]-1, :]
    
    return y

In [17]:
nan_cols = np.asarray(nan_cols)
a = 0

if nan_cols.size > 0:
    if len(np.where(nan_cols==nan_cols[0])[0])*3 == len(nan_cols):
        all_trajectories = np.zeros((trajectory.shape[0]*int(len(trajectories)), trajectory.shape[1]-len(nan_cols)))
else:
    all_trajectories = np.zeros((trajectory.shape[0]*int(len(trajectories)), trajectory.shape[1]))

for i, time_bin in enumerate(trajectories):
    if time_bin != 'behavs':
        trajectory = loadmat(path + time_bin)
        trajectory = trajectory['trajectory'] 
        if nan_cols.size > 0:
            trajectory = np.delete(trajectory, nan_cols, 1)
            
        idx_2 = i*trajectory.shape[0]
        window = f_s
        mov_av_traj = rolling_mean(trajectory, window)
        all_trajectories[idx_2:idx_2+trajectory.shape[0], 0:all_trajectories.shape[1]] = mov_av_traj
        
# convert nan to number when not it is a sparse recurrence (not an entire column)
all_trajectories = np.nan_to_num(all_trajectories)
path_out = '/rds/general/user/lr4617/home/4th_Year_Project/CAPTURE_rat_multidimensional/raw_data/' + str(mode) + '/traj_' + str(traj_n) + '/trajectory.npy'
np.save(path_out, all_trajectories)
print('Data points in standard trajectory: ', all_trajectories.shape)

Data points in standard trajectory:  (3348000, 60)
